In [ ]:
import torch.nn as nn
import numpy as np
import gym
from seagul.rl.ars.meta_ars import MetaARSAgent
import matplotlib.pyplot as plt
import torch
import dill
import seagul.envs
import pybullet_envs
import pickle
from scipy.signal import find_peaks

from common import *

%load_ext autoreload
%autoreload 2

from IPython.display import Audio
sound_file = '/home/sgillen/Downloads/chime.wav'
#Audio(filename=sound_file, autoplay=True)

In [ ]:
meta_agent = pickle.load(open("agents/meta_hmap_hopper_flat4", "rb"))
agent0 = meta_agent.agents[4]

In [ ]:
env_name = "hmap_hopper-v0"
seed = 4
env = gym.make(env_name,slope_set=[0], random=True)
#env = gym.make("Hopper-v2")

#agent = meta_agent.agents[seed]
obs, act, rew, _ = do_rollout(env, agent0.model, render=False, ep_length=1000)
print(sum(rew))
plt.plot(obs);

In [ ]:
import copy
slope_set = [-.01, 0.0, .01]
agent = copy.deepcopy(agent0)
#agent.postprocessor = mdim_div
agent.env_config = {"slope_set":slope_set, "random":True}
agent.learn(200)
agent_list.append(agent)
    
Audio(filename=sound_file, autoplay=True)

In [ ]:
import copy
agent_list = []
slope_set = [-.02, 0.0, .02]
for slope in slope_set:
    agent = copy.deepcopy(agent0)
    #agent.postprocessor = mdim_div
    agent.env_config = {"slope_set":[slope], "random":False}
    agent.learn(200)
    agent_list.append(agent)
    
Audio(filename=sound_file, autoplay=True)

In [ ]:
import xarray as xr

n_runs = 5
data = xr.DataArray(np.zeros((len(slope_set), len(agent_list), n_runs)),
                    dims=("slope", "agent", "run"))

for i, slope in enumerate(slope_set):
    env = gym.make(env_name, slope_set=[slope], random=False)
    for j, agent in enumerate(agent_list):
        for run in range(n_runs):
            obs,act,rew,_ = do_rollout(env, agent.model)
            data.loc[i, j, run] = sum(rew).item()
            

In [ ]:
data.mean(dim='run')

In [ ]:
from seagul.nn import fit_model, MLP
net = MLP(12, 3, 2, 32, input_bias=True)

loss_hist = fit_model(net, X, Y, 25, batch_size=2048, loss_fn=torch.nn.CrossEntropyLoss(),use_tqdm=False)

plt.plot(loss_hist)

In [ ]:
print(X[:,-1])
net(X)

In [ ]:
#pols = [agent.model for agent in agent_list]
pols = [agent.model for agent in agent_list]
#npol.state_std[-1] = 1
npol = copy.deepcopy(pols[1])
npol.state_std[-1] = .01
def do_nn_rollout_from_state(env, pols, init_state, obs_std=0, act_std=0, num_steps=1000, render=True):
    torch.autograd.set_grad_enabled(False)

    act_list = []
    obs_list = []
    rew_list = []
    val_list = []

    obs = my_reset(env, init_state)
    done = False
    obs_size = env.observation_space.shape[0]
    act_size = env.action_space.shape[0]
    pi = 0; old_pi = pi
    policy = pols[pi]

    while not done:
        if render:
            env.render()
            time.sleep(.01)
            
        n_obs = (obs - npol.state_mean)/npol.state_std
    
        preds = net(np.array(n_obs,dtype=np.float32))
        pi = torch.argmax(preds)
#         if n_obs[-1] < -tol:
#             pi = 0
#         elif -tol <= n_obs[-1] <= tol:
#             pi = 1
#         elif  n_obs[-1] >= tol:
#             pi = 2
#         else:
          #  panic
        
        if pi != old_pi:
            print(f"went from {old_pi} to {pi}")

        #print(env.get_height(0), env.get_height(1), env.get_height(2), obs[-1])
        old_pi = pi
        #policy = pols[pi]

        policy = pols[pi]

        obs += np.random.randn(obs_size)*obs_std
        obs_list.append(np.copy(obs))
        
        
        act,_,_,_ = policy.step(obs)
        act += np.random.randn(act_size)*act_std
        obs, rew, done, _ = env.step(act)
        
        act_list.append(np.copy(act))
        rew_list.append(rew)
        
    ep_obs = np.stack(obs_list)
    ep_act = np.stack(act_list)
    ep_rew = np.array(rew_list).reshape(-1,1)
    ep_vals = np.array(val_list).reshape(-1,1)

    
    torch.autograd.set_grad_enabled(True)
    return ep_obs, ep_act, ep_rew, ep_vals

#mesh_map = {state.id:state for state in mesh.values()}

env.unwrapped.slope_set = [-.01, 0.0, 0.01]
#env.unwrapped.slope_set = [0.0]

env.unwrapped.random=True
obs,act,rews,vals = do_nn_rollout_from_state(env, pols, env.reset(), render=True)

In [ ]:
print(sum(rews))
plt.plot(((obs - npol.state_mean)/npol.state_std)[:,-1]);

In [ ]:
plt.plot(net(np.array(obs,dtype=np.float32)).detach())
plt.legend([0,1,2])

In [ ]:
X2 = (obs - npol.state_mean)/npol.state_std
X2 = np.array(X2,dtype=np.float32)
tol = 0.1
#X = [(point.state - npol.state_mean)/npol.state_std for point in mesh_map.values()]

Yl = []
for x in X2:
    if x[-1] < -tol:
        Yl.append(0)
    elif -tol <= x[-1] <= tol:
        Yl.append(1)
    elif  x[-1] >= tol:
        Yl.append(2)
    else:
        panic
        
    

Y2 = np.stack(Yl)
Y2 = np.array(Y2,dtype=np.long)

In [ ]:
loss_hist = fit_model(net, X2, Y2, 10, batch_size=512, loss_fn=torch.nn.CrossEntropyLoss(),use_tqdm=False)


In [ ]:
Y2

In [ ]:
((obs - npol.state_mean)/npol.state_std)[:,-1]

In [ ]:
X2[:,-1]